In [12]:
import pandas as pd
import numpy as np
from epiweeks import Week
import dask.dataframe as dd
from datetime import datetime

In [13]:
dic_sex = {'MASCULINO': 1,'FEMENINO':2}
dic_met = {'PR': 1, 'PCR':2, 'AG':3}
dic_dep = {
    "AMAZONAS"	:1,
    "ANCASH"	:2,
    "APURIMAC"	:3,   
    "AREQUIPA"	:4,
    "AYACUCHO"	:5,   
    "CAJAMARCA"	:6,
    "CALLAO"	:7,
    "CUSCO"	    :8,
    "HUANCAVELICA":9,	
    "HUANUCO"	:10,	
    "ICA"	    :11,	
    "JUNIN"	    :12,
    "LA LIBERTAD":13,
    "LAMBAYEQUE":14,
    "LIMA"	    :15,
    "LORETO"	:16,
    "MADRE DE DIOS":17,
    "MOQUEGUA"	:18,	
    "PASCO"	    :19,
    "PIURA"	    :20,
    "PUNO"	    :21,
    "SAN MARTIN":22,
    "TACNA"	    :23,
    "TUMBES"	:24,
    "UCAYALI"	:25
}

In [14]:
# Leer dataset
url_cas = 'RawData/positivos_covid.csv'
col_cas = ['FECHA_RESULTADO','EDAD', 'DEPARTAMENTO', 'SEXO', 'METODODX']
df_cas = pd.read_csv(url_cas, sep = ';', usecols = col_cas, dtype={'EDAD': 'float64', 
                                                                   'FECHA_RESULTADO': 'float64'})

In [ ]:
# Correguir datos erróneos encontrados en el anáisis exploradorio de "0_ExploratoryDataAnalysis.ipynb"
df_cas = df_cas.dropna()
df_cas = df_cas.astype({'EDAD':'int8', 'FECHA_RESULTADO': 'int32'})

df_cas.drop(df_cas[df_cas.SEXO == '.'].index, inplace = True)
df_cas.drop(df_cas[df_cas.EDAD == -79].index, inplace = True)  # un -9 en edad

# Categorizar edad en grupos etáreos
def edad_cat(df, col):
    """Función para categorizar edad en grupos etáreos"""
    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]
    return np.select(conditions, choices, default=np.nan).astype('int8')
df_cas['edad_cat'] = edad_cat(df_cas, 'EDAD') 

# Reemplazar sexo con un code
df_cas['SEXO'].replace(dic_sex, inplace=True)
df_cas['SEXO'] = df_cas['SEXO'].astype('int8')

# Ordenar columnas
df_cas.rename(columns = {'FECHA_RESULTADO':'fecha_resultado',
                         'EDAD': 'edad',
                         'SEXO': 'sexo',
                         'DEPARTAMENTO': 'departamento',
                         'METODODX': 'metododx'}, inplace=True)
df_cas = df_cas[['id_persona','fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento']] \
         .sort_values(by = ['id_persona']) \
         .reset_index(drop = True)

# Codificar variables
df_cas['departamento'] = df_cas['departamento'].map(dic_dep).astype('int8')  
df_cas['metododx'].replace(dic_met, inplace=True)
df_cas['metododx'] = df_cas['metododx'].astype('int8') 

# Transformar a formato fecha
df_cas.loc[:,'fecha_resultado'] = pd.to_datetime(df_cas['fecha_resultado'], format = '%Y%m%d')

# Agregar la diferencia en días desde el primer caso
fecha_min = df_cas['fecha_resultado'].min()

def days_between(d1, d2):
    return abs((d2 - d1).days)

df_cas['time_days'] = df_cas['fecha_resultado'].apply(lambda row : days_between(row, fecha_min))

# Agregar la semana epidemiológica y el año
df_cas[['year','epi_week']] = df_cas['fecha_resultado'].apply(lambda date : Week.fromdate(date) \
                                                                                  .weektuple()) \
                                                                                  .tolist()
# Reorder
df_cas = df_cas[['fecha_resultado','year', 'epi_week', 'time_days', 
                  'sexo', 'edad', 'edad_cat', 'departamento', 'metododx']] \
         .sort_values(by = ['fecha_resultado']) \
         .reset_index(drop = True)

In [15]:
df_cas

,fecha_resultado,year,epi_week,time_days,sexo,edad,edad_cat,departamento,metododx
0,2020-03-06,2020,10,0,1,25,1,15,2
1,2020-03-07,2020,10,1,1,7,0,15,2
2,2020-03-07,2020,10,1,1,25,1,15,2
3,2020-03-07,2020,10,1,1,29,1,4,2
4,2020-03-07,2020,10,1,2,74,6,15,2
...,...,...,...,...,...,...,...,...,...
2298847,2022-01-06,2022,1,671,1,31,2,13,2
2298848,2022-01-06,2022,1,671,1,31,2,6,2
2298849,2022-01-06,2022,1,671,2,11,0,15,2
2298850,2022-01-06,2022,1,671,2,43,3,15,3


In [16]:
df_cas.to_csv('Data/DP5_covid19-peru_info-casos-positivos.csv')

***

In [47]:
# Leer el dataset
url_fal = 'RawData/fallecidos_covid.csv'
col_fal = ['FECHA_FALLECIMIENTO', 'EDAD_DECLARADA', 'SEXO', 'DEPARTAMENTO']
df_fal = pd.read_csv(url_fal, sep = ';', usecols = col_fal, dtype = {'FECHA_FALLECIMIENTO':'int32', 
                                                                     'EDAD_DECLARADA': 'int8'})

In [48]:
# Correguir datos erróneos encontrados en el anáisis exploradorio de "0_ExploratoryDataAnalysis.ipynb"
df_cas = df_cas.dropna()

# Ordenar columnas
df_fal.rename(columns = {'FECHA_FALLECIMIENTO':'fecha_fallecimiento', 'EDAD_DECLARADA': 'edad',
                         'SEXO': 'sexo', 'DEPARTAMENTO': 'departamento'}, inplace=True)

df_fal.drop(df_fal[df_fal.sexo == '.'].index, inplace = True)
df_fal.drop(df_fal[df_fal.edad == -9].index, inplace = True)  # un -9 en edad

# Transformar a formato fecha
df_fal.loc[:,'fecha_fallecimiento'] = pd.to_datetime(df_fal['fecha_fallecimiento'], format = '%Y%m%d')

# Convertir edad en categoría
df_fal['edad_cat'] = edad_cat(df_fal, 'edad') 

# Reemplazar sexo con un code
df_fal['sexo'] = df_fal['sexo'].replace(dic_sex).astype('int8')

# Codificar variables
df_fal['departamento'] = df_fal['departamento'].map(dic_dep).astype('int8')  

# Transformar a formato fecha
df_fal.loc[:,'fecha_fallecimiento'] = pd.to_datetime(df_fal['fecha_fallecimiento'], format = '%Y%m%d')

# Agregar la diferencia en días desde el primer caso
fecha_min = df_fal['fecha_fallecimiento'].min()

df_fal['time_days'] = df_fal['fecha_fallecimiento'].apply(lambda row : days_between(row, fecha_min))

# Agregar la semana epidemiológica y el año
df_fal[['year','epi_week']] = df_fal['fecha_fallecimiento'].apply(lambda date : Week.fromdate(date) \
                                                                                  .weektuple()) \
                                                                                  .tolist()
# Reorder
df_fal = df_fal[['fecha_fallecimiento','year', 'epi_week', 'time_days', 
                  'sexo', 'edad', 'edad_cat', 'departamento']] \
         .sort_values(by = ['fecha_fallecimiento']) \
         .reset_index(drop = True)

In [49]:
df_fal

,fecha_fallecimiento,year,epi_week,time_days,sexo,edad,edad_cat,departamento
0,2020-03-03,2020,10,0,1,63,5,22
1,2020-03-04,2020,10,1,1,85,7,22
2,2020-03-07,2020,10,4,2,24,1,16
3,2020-03-11,2020,11,8,2,78,6,20
4,2020-03-14,2020,11,11,1,4,0,15
...,...,...,...,...,...,...,...,...
202971,2022-01-06,2022,1,674,1,93,7,15
202972,2022-01-06,2022,1,674,2,82,7,14
202973,2022-01-06,2022,1,674,1,78,6,23
202974,2022-01-06,2022,1,674,2,70,6,11


In [50]:
df_cas.to_csv('Data/DP6_covid19-peru_info-fallecidos.csv')